In [87]:
import os 
import openai 
import sys 


In [88]:
from dotenv import load_dotenv
load_dotenv()

# Get API key and model name from environment
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_model = os.getenv("OPENAI_MODEL")

In [89]:
os.environ['OPENAI_API_KEY'] = openai_api_key

In [90]:
from langchain.document_loaders import PyPDFLoader

In [91]:
loaders = [
    PyPDFLoader(r"C:\Users\karti\Desktop\Project\openai_learning\docs\Triplet_loss_original_paper.pdf"),
    PyPDFLoader(r"C:\Users\karti\Desktop\Project\openai_learning\docs\yolo.pdf")
]

In [92]:
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [93]:
print(docs)

[Document(metadata={'source': 'C:\\Users\\karti\\Desktop\\Project\\openai_learning\\docs\\Triplet_loss_original_paper.pdf', 'page': 0, 'page_label': '1'}, page_content='Accepted as a workshop contribution at ICLR 2015\nDEEP METRIC LEARNING USING TRIPLET NETWORK\nElad Hoffer\nDepartment of Electrical Engineering\nTechnion Israel Institute of Technology\nehoffer@tx.technion.ac.il\nNir Ailon∗\nDepartment of Computer Science\nTechnion Israel Institute of Technology\nnailon@cs.technion.ac.il\nABSTRACT\nDeep learning has proven itself as a successful set of models for learning useful\nsemantic representations of data. These, however, are mostly implicitly learned as\npart of a classiﬁcation task. In this paper we propose the triplet network model,\nwhich aims to learn useful representations by distance comparisons. A similar\nmodel was deﬁned by Wang et al. (2014), tailor made for learning a ranking\nfor image information retrieval. Here we demonstrate using various datasets that\nour model 

In [94]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [95]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [96]:
splits = text_splitter.split_documents(docs)

In [97]:
len(docs)

18

In [98]:
len(splits)

55

In [99]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [100]:
sentence1 = "The weather is great"
sentence2 = "Is the weather is great"
sentence3 = "The weather is awesome"

In [101]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [102]:
print(embedding1)
print(embedding2)
print(embedding3)

[0.008669976152892342, 0.005950354510727025, 0.008070522926616815, -0.030515285236497273, -0.012083699937693336, 0.023359715959754314, -0.00644569146100156, -0.03541186455123621, 0.00658135722056884, -0.036825310472270265, -0.006884238223269687, 0.01737781243112229, -0.007805502049253575, -0.020318282786558062, -0.0009133762804693336, -0.013743236673421828, 0.031146287946339085, -0.001209947514513777, 0.021353127454216053, 0.0048460997685038495, 0.010701804431548148, 0.02205985041473308, 0.008404954344206525, -0.005404537082894824, -0.010077111432155075, 0.0016390292454475015, 0.000591564924061379, -0.021113347281292934, 0.013781096798759433, -0.012733632710203955, 0.020166842285207636, -0.0012809352495217878, -0.0259468276288993, -0.02294325458104065, -0.006928408524717322, -0.026300188177835238, -0.020229943114985365, -0.005278336820323234, -0.0052373213740999685, 0.0004590543980682671, 0.008909756325815463, -0.006657077471244051, 0.009029647343599597, 0.0028899919733538994, -0.00861

In [103]:
import numpy as np


In [104]:
np.dot(embedding1,embedding2)

np.float64(0.9725742132852606)

In [105]:
np.dot(embedding1,embedding3)

np.float64(0.9707327325149188)

In [106]:
np.dot(embedding2,embedding3)

np.float64(0.9465832970916502)

chroma refer to open source vector database specially designed for storing and retrieving vector embeddings 

In [107]:
from langchain.vectorstores import Chroma

In [108]:
persist_directory = r'C:\Users\karti\Desktop\Project\openai_learning\vector_database\Learning'

In [109]:
vectordb = Chroma.from_documents(
    documents = splits,
    embedding = embedding,
    persist_directory=persist_directory
)

In [110]:
print(vectordb._collection.count())

110


In [111]:
question = "can you tell me about the how to optimize the yolo"

In [112]:
docs = vectordb.similarity_search(question , k = 3)

In [113]:
print(docs)

[Document(metadata={'page': 0, 'page_label': '1', 'source': 'C:\\Users\\karti\\Desktop\\Project\\openai_learning\\docs\\yolo.pdf'}, page_content='means we can process streaming video in real-time with\nless than 25 milliseconds of latency. Furthermore, YOLO\nachieves more than twice the mean average precision of\nother real-time systems. For a demo of our system running\nin real-time on a webcam please see our project webpage:\nhttp://pjreddie.com/yolo/.\nSecond, YOLO reasons globally about the image when\n1\narXiv:1506.02640v5  [cs.CV]  9 May 2016'), Document(metadata={'page': 0, 'page_label': '1', 'source': 'C:\\Users\\karti\\Desktop\\Project\\openai_learning\\docs\\yolo.pdf'}, page_content='means we can process streaming video in real-time with\nless than 25 milliseconds of latency. Furthermore, YOLO\nachieves more than twice the mean average precision of\nother real-time systems. For a demo of our system running\nin real-time on a webcam please see our project webpage:\nhttp://pjre

In [114]:
len(docs)

3

In [115]:
print(docs[0].page_content)

means we can process streaming video in real-time with
less than 25 milliseconds of latency. Furthermore, YOLO
achieves more than twice the mean average precision of
other real-time systems. For a demo of our system running
in real-time on a webcam please see our project webpage:
http://pjreddie.com/yolo/.
Second, YOLO reasons globally about the image when
1
arXiv:1506.02640v5  [cs.CV]  9 May 2016


In [116]:
vectordb.persist()

In [117]:
question = "Tell me the advantage of using simense network"

In [118]:
docs = vectordb.similarity_search(question , k = 5)

In [119]:
docs[0].page_content

'Siamese networks are also sensitive to calibration in the sense that the notion of similarity vs dis-\nsimilarity requires context. For example, a person might be deemed similar to another person when\na dataset of random objects is provided, but might be deemed dissimilar with respect to the same\nother person when we wish to distinguish between two individuals in a set of individuals only. In\nour model, such a calibration is not required. In fact, in our experiments here, we have experienced\nhands on the difﬁculty in using Siamese networks.\nWe follow a similar task to that of Chechik et al. (2010). For a set of samples P and a chosen rough\nsimilarity measure r(x, x′) given through a training oracle (e.g how close are two images of objects\n∗The author acknowledges the generous support of ISF grant number 1271/13\n1\narXiv:1412.6622v4  [cs.LG]  4 Dec 2018'

In [120]:
docs[1].page_content

'Siamese networks are also sensitive to calibration in the sense that the notion of similarity vs dis-\nsimilarity requires context. For example, a person might be deemed similar to another person when\na dataset of random objects is provided, but might be deemed dissimilar with respect to the same\nother person when we wish to distinguish between two individuals in a set of individuals only. In\nour model, such a calibration is not required. In fact, in our experiments here, we have experienced\nhands on the difﬁculty in using Siamese networks.\nWe follow a similar task to that of Chechik et al. (2010). For a set of samples P and a chosen rough\nsimilarity measure r(x, x′) given through a training oracle (e.g how close are two images of objects\n∗The author acknowledges the generous support of ISF grant number 1271/13\n1\narXiv:1412.6622v4  [cs.LG]  4 Dec 2018'

# Retrival 

In [121]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [122]:
texts

['The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).',
 'A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.',
 'A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.']

In [123]:
smalldb = Chroma.from_texts(texts, embedding)

In [124]:
smalldb

In [125]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [126]:
smalldb.similarity_search(question , k = 2)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.')]

In [127]:
smalldb.max_marginal_relevance_search(question, k = 2 ,fetch_k=3)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).')]

In [128]:
import langchain.llms
print(dir(langchain.llms))

['Any', 'BaseLLM', 'Callable', 'Dict', 'LangChainDeprecationWarning', 'Type', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__getattr__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_import_ai21', '_import_aleph_alpha', '_import_amazon_api_gateway', '_import_anthropic', '_import_anyscale', '_import_arcee', '_import_aviary', '_import_azure_openai', '_import_azureml_endpoint', '_import_baidu_qianfan_endpoint', '_import_bananadev', '_import_baseten', '_import_beam', '_import_bedrock', '_import_bittensor', '_import_cerebriumai', '_import_chatglm', '_import_clarifai', '_import_cohere', '_import_ctransformers', '_import_ctranslate2', '_import_databricks', '_import_databricks_chat', '_import_deepinfra', '_import_deepsparse', '_import_edenai', '_import_fake', '_import_fireworks', '_import_forefrontai', '_import_gigachat', '_import_google_palm', '_import_gooseai', '_import_gpt4all', '_import_gradient_ai', '_import_huggingface_endpoint', '_import_huggingf

In [129]:
from langchain_openai import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo


In [130]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description=r"The knowledge about the simense network 'C:\Users\karti\Desktop\Project\openai_learning\docs\Triplet_loss_original_paper.pdf",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [133]:

document_content_description = "Lecture notes"
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

ImportError: Cannot import lark, please install it with 'pip install lark'.